# Purpose of this notebook
The goal of this notebook is to introduce you to the Python interface to Neuroglancer. There are several advantages for using the Python interface over using the visualization links generated at the braincogs00.pni.princeton.edu site. The main advantage is that it gives you programmatic control over your Neuroglancer session. This helps for making reproducible figures via the screenshot feature. It also allows you to keep the Neuroglancer session open indefinitely, whereas the links on braincogs00 expire after a few hours of inactivity. If you are making annotations and that takes longer than a single sitting, you could lose your progress if the Neuroglancer session is closed. This notebook prevents that from happening. 

Here is what we will cover in this notebook:
- Start a Neuroglancer session from Python and load in public data 
- Manipulate the Neuroglancer session from Python
- Configure a screenshot reproducibly
- Load in your private light-sheet data

# Setup
In order to run the code in this notebook, you will need a conda environment with python3 and containing some additional libraries. This environment, which I call "ng" below but you could call whatever you want, can be set up in the following way:

In terminal:
- conda create -n ng python=3.8 -y
- conda activate ng # (or source activate ng, depending on which version of conda you have)
- pip install cloud-volume
- pip install neuroglancer <br>

\# To enable you to use jupyter notebooks to work with this environment as a kernel:
In terminal:
- conda activate ng
- pip install --user ipykernel
- python -m ipykernel install --user --name=ng

Once this is all installed, make sure to select this conda environment as the kernel when running this notebook via Kernel -> Change Kernel (you may have to restart the jupyter notebook server if you just created the conda environment)

In [80]:
import neuroglancer
import numpy as np

#  Start a Neuroglancer session from Python and load in public data 

In [2]:
# Set which client you want to use 
# This uses the BRAINCOGS client to get the latest features.
neuroglancer.set_static_content_source(url='https://neuroglancer-braincogs.appspot.com')

In [151]:
# Make a viewer object that represents your connection to a new Neuroglancer session
viewer = neuroglancer.Viewer()

In [152]:
# Load in the Princeton Mouse Brain Atlas 
# This is hosted publicly in Google Cloud -- hence the "gs://" in the source address below
# wanglab-pma is our public Google Cloud bucket where we host the atlases
 
with viewer.txn() as s:
    s.layers["Princeton Mouse Atlas"] = neuroglancer.SegmentationLayer(
        source='precomputed://gs://wanglab-pma/princetonmouse',
    )
print(viewer)

http://127.0.0.1:40505/v/d3fc7122655ddc9aa15c342459779206fe181654/


Click the link above and it should bring you to a Neuroglancer session with the Princeton Mouse Brain Atlas loaded. That link will be active for as long as this jupyter notebook session is active. 

# Manipulate the Neuroglancer session from Python
Pretty much all of the things you can do in the browser with your mouse and keyboard you can also do programmatically from Python. After each code cell look back at your Neuroglancer session to see the changes that were made. I recommend arranging the windows so you can see this jupyter notebook and the Neuroglancer window simultaneously. Here are some examples:

In [153]:
# Show the entire configuration of the viewer -- these are all things you can change
viewer.state

ViewerState({"dimensions": {"x": [2e-05, "m"], "y": [2e-05, "m"], "z": [2e-05, "m"]}, "position": [176.5, 320.5, 270.5], "crossSectionScale": 1, "projectionScale": 1024, "layers": [{"type": "segmentation", "source": "precomputed://gs://wanglab-pma/princetonmouse", "name": "Princeton Mouse Atlas"}], "layout": "4panel"})

In [154]:
# Zoom way out
with viewer.txn() as s:
    s.crossSectionScale = 8

In [155]:
# Zoom back in
with viewer.txn() as s:
    s.crossSectionScale = 1

In [156]:
# Change the position you are looking at
with viewer.txn() as s:
    s.position = [135,514,144] # Somewhere in Crus I in the Cerebellum

In [157]:
# Only look at sagittal planes
with viewer.txn() as s:
    s.layout = 'xy'

In [158]:
# Select segments by ID 
with viewer.txn() as s:
    s.layers["Princeton Mouse Atlas"].layer.segments = {512, 1025, 1033, 1041, 153, 
                                  1049, 1056, 1064, 936, 944, 951,
                                  957, 1091, 968, 78, 846, 976,
                                  728, 984, 989, 96, 354,
                                  101, 744, 1134, 1007} # only the Cerebellum 

In [159]:
# Open the right hand control panel and show the selected segments
with viewer.txn() as s:
    s.selectedLayer.layer = "Princeton Mouse Atlas"
    s.selectedLayer.visible = True
    s.layers["Princeton Mouse Atlas"].tab = "segments"
    s.selectedLayer.size=500 # sets the width of the right hand panel

In [160]:
# Decrease saturation
with viewer.txn() as s:
    s.layers[0].saturation = 0.5

In [161]:
# Change color of one of the segments
with viewer.txn() as s:
    s.layers["Princeton Mouse Atlas"].segment_colors[1007] = "#a83c32"

In [162]:
# Merge all of the segments shown so they all have the same color
with viewer.txn() as s:
    s.layers["Princeton Mouse Atlas"].equivalences = [(512, 1025, 1033, 1041, 153, 
                                  1049, 1056, 1064, 936, 944, 951,
                                  957, 1091, 968, 78, 846, 976,
                                  728, 984, 989, 96, 354,
                                  101, 744, 1134, 1007)]

In [163]:
# Undo the merge and reset the segments
with viewer.txn() as s:
    s.layers["Princeton Mouse Atlas"].equivalences = []
    s.layers["Princeton Mouse Atlas"].layer.segments = {512, 1025, 1033, 1041, 153, 
                                  1049, 1056, 1064, 936, 944, 951,
                                  957, 1091, 968, 78, 846, 976,
                                  728, 984, 989, 96, 354,
                                  101, 744, 1134, 1007}

In [164]:
# Toggle the yellow bounding box, RGB coordinate axes, scale bar
with viewer.txn() as s:
    s.showDefaultAnnotations = False # turn off yellow bounding box
    s.show_axis_lines=False # turn off axes
    s.show_scale_bar = False

In [165]:
# Change background color
with viewer.txn() as s:
    s.cross_section_background_color = "#000000" # black

In [166]:
# Take a screenshot using the webdriver
from neuroglancer import webdriver as webd

In [167]:
# Start the webdriver which should open a new window which clones your viewer
webdriver = webd.Webdriver(viewer, headless=False,)

http://127.0.0.1:40505/favicon.ico - Failed to load resource: the server responded with a status of 404 (Not Found)
https://www.googleapis.com/storage/v1/b/wanglab-pma/o/princetonmouse%2Fmesh_mip_0_err_40%2Finfo?alt=media - Failed to load resource: the server responded with a status of 404 ()


If you get errors starting the webdriver you may need to do "pip install selenium" into your ng conda environment. You may also need to download the "chromedriver" program if you are using Google Chrome. https://chromedriver.chromium.org/downloads. Put the chromedriver exectuable in your path and then retry running this notebook

In [168]:
webdriver.driver.set_window_size(1200,800)

In [169]:
# Hide the top control bar and layer names
with viewer.config_state.txn() as s:
    s.show_ui_controls = False
#     s.show_panel_borders = True
#     s.show_layer_panel = True
#     s.show_help_button= True
#     s.show_location= True

In [170]:
# Take the screenshot of the webdriver window
screenshot_filename = './test_screenshot.png'
webdriver.driver.save_screenshot(screenshot_filename)

True

In [172]:
# Create a publicly sharable URL with the current viewer state
neuroglancer.to_url(viewer.state,prefix="https://neuroglancer-braincogs.appspot.com")

'https://neuroglancer-braincogs.appspot.com#!%7B%22dimensions%22:%7B%22x%22:%5B2e-05,%22m%22%5D,%22y%22:%5B2e-05,%22m%22%5D,%22z%22:%5B2e-05,%22m%22%5D%7D,%22position%22:%5B135,514,144%5D,%22crossSectionScale%22:1,%22projectionScale%22:1024,%22layers%22:%5B%7B%22type%22:%22segmentation%22,%22source%22:%22precomputed://gs://wanglab-pma/princetonmouse%22,%22tab%22:%22segments%22,%22saturation%22:0.5,%22segmentColors%22:%7B%2278%22:%22#a83c32%22%7D,%22segments%22:%5B%221007%22,%22101%22,%221025%22,%221033%22,%221041%22,%221049%22,%221056%22,%221064%22,%221091%22,%221134%22,%22153%22,%22354%22,%22512%22,%22728%22,%22744%22,%2278%22,%22846%22,%22936%22,%22944%22,%22951%22,%22957%22,%2296%22,%22968%22,%22976%22,%22984%22,%22989%22%5D,%22name%22:%22Princeton%20Mouse%20Atlas%22%7D%5D,%22showAxisLines%22:false,%22showScaleBar%22:false,%22showDefaultAnnotations%22:false,%22selectedLayer%22:%7B%22layer%22:%22Princeton%20Mouse%20Atlas%22,%22visible%22:true,%22size%22:500%7D,%22crossSectionBackgrou

Copy the above link into a new tab in your browser -- the result should be identical to our first link. Note that this is a static link - if you make more changes to the viewer you will have to regenerate the link.  

# Load your private light-sheet data into Neuroglancer
So far we have been using a public dataset (the Princeton Mouse Atlas) that is hosted in the cloud. To view private data in Neuroglancer (such as your light-sheet data that lives on bucket), you need to host it yourself (or move it to Google Cloud Storage). Fortunately there is a python package called "cloud-volume" that makes hosting your private data easy. You should have already installed this into your "ng" conda environment as per the instructions at the beginning of the notebook.

Before your data can be hosted, they need to have been converted to the "precomputed" format. This is part of the light-sheet pipeline that we run for you, so if you have a request at braincogs00.pni.princeton.edu we more than likely have already done this for your data. We store the precomputed data in the "viz/" subfolder for each of your samples on bucket. This path can be found for your request/sample under:

```python
/jukebox/LightSheetData/lightserv/{netid}/{request_name}/{sample_name}/imaging_request_1/viz
```
Replace {netid}, {request_name}, {sample_name} with your information to find your data.

The precomputed data folder for your raw data are stored in a `raw/` subfolder and a `processing_request_1/` subfolder for the other products, like blended and atlas-registered volumes. For example the precomputed layer for blended channel 488 data the filepath would be:

```python
/jukebox/LightSheetData/lightserv/{netid}/{request_name}/{sample_name}/imaging_request_1/viz/processing_request_1/blended/channel_488/channel488_blended
```
 
The precomputed data folders have file structures that looks like this:
```
├── 14400_14400_2000
├── 1800_1800_2000
├── 28800_28800_2000
├── 3600_3600_2000
├── 57600_57600_2000
├── 7200_7200_2000
├── info
└── provenance
```
The key thing is that there needs to be an `info` file inside the precomputed folder. That is how you know you have found the right folder.

If you are not sure whether we have created the precomputed layer for you yet, or you cannot find a folder that has contents like what is shown above, please contact lightservhelper@gmail.com or use the #lightsheet-software slack channel for help. Note that if you know how to use braincogs00.pni.princeton.edu to visualize your data (see: https://braincogs00.pni.princeton.edu/FAQ if you don't), when you generate the Neuroglancer link there is a table that shows the filepaths to your precomputed layers once you have filled out the visualization setup form.

Once you have found the path to your precomputed data, you  need to host it. To do this, open up a Python or iPython session outside of this jupyter notebook and run the following code, but change the `layer_dir` to your light sheet data precomputed layer folder on bucket that you found above.  

```python
from cloudvolume import CloudVolume
layer_dir = "ENTER THE PATH TO YOUR LAYER HERE"
vol = CloudVolume(f'file://{layer_dir}')
vol.viewer(port=1338)
```

**Note:** Make sure to have the ng conda environment activated before running python, otherwise you will get some "module not found" errors. You don't want to run this code in this notebook because it will cause it to hang and you won't be able to run the rest of the cells.

Once your data are hosted you can make a new Neuroglancer python instance and load them in like this:

In [17]:
# Makes a new viewer object that is separate from the one we were working with above 
viewer = neuroglancer.Viewer()

In [18]:
layer_dir = "ENTER THE PATH TO YOUR LAYER HERE" # use the same path as you used when you hosted the data 
layer_name = "MY CHANNEL NAME" # change this to whatever you want. 
with viewer.txn() as s:
    s.layers[layer_name] = neuroglancer.ImageLayer(
        source='precomputed://http://localhost:1338',
    )
print(viewer)

http://127.0.0.1:42971/v/9cd83e3d6102ab4837dfb8700ba3a4b1ded832db/


Click that link and it will bring you to Neuroglancer with your data loaded in.

You might notice that the contrast is poor. To change that execute this cell:

In [21]:
# Change image contrast so the data are more visible
# This inverts the color map (the 1.0- part) and increases the contrast (the factor of 300). 
# Use the d and f keys in the browser to update the contrast to your liking
with viewer.txn() as s:
    imagelayer = s.layers[layer_name]
    imagelayer.shader = """ void main() {emitGrayscale(1.0-toNormalized(getDataValue())*300.0);} """